# Generate Period difference and SD

Generating files for difference between early and late period in SSTA, then standard deviation in SSTA between early, late, full periods

In [1]:
%reset -f

In [2]:
%load_ext autoreload

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import sys

sys.path.append('/home/hbyrne/Research/Tools')

#### Libraries

In [15]:
import numpy as np
import pandas as pd
import xarray as xr
import os

from Utils_Functions_HB import SelectModelInputs
from GradientProjectFunctions import ClassifyHistModelsLite, MakeChangeDir, RemoveClimatology
from GradTrendClasses import ModelInput

#### Directories

In [7]:
# inputs
dirSummaries = '/home/hbyrne/Research/Gradient_project/gradient_project/Outputs/Summaries'

# outputs
dirSSTPeriods = '/home/hbyrne/Research/Gradient_project/gradient_project/Outputs/MeanSST_Periods'
dirSDPeriods = '/home/hbyrne/Research/Gradient_project/gradient_project/Outputs/SD_Periods'

### Calculation

In [10]:
os.chdir(dirSummaries)
fileName = 'ModelSummary_Integral.csv'

modelSummary = pd.read_csv(fileName, index_col=0)

In [11]:
# defining the region
lonmin, lonmax = 140, 280
latmin, latmax = -10, 10

In [12]:
# sort the models by range
modelsSorted = modelSummary.sort_values('Weighted Range').index

# choosing the two intervals
periodStart = ('1850', '1870')
periodEnd = ('1990', '2010')

# choosing the number of runs
N = 10

In [ ]:
# initialise the dictionaries

# difference in SST
dictPeriod1 = {}
dictPeriod2 = {}
dictDiff = {}

# std for different periods
dictSD1 = {}
dictSD2 = {}
dictSDFull = {}

for model in modelsSorted:
    
    # first selecting the runs
    # inputting the historical models based on this
    modelListHist, modelListScenario = SelectModelInputs(models = [model])

    # checking that they span the full period
    histModels = ClassifyHistModelsLite(modelListHist)

    i = 0
    for classification, item in histModels.items():

        if classification == 'Full':
            for modelName in item:
                if i < N:

                    # opening the dataset
                    da = ModelInput(modelName).ds.ts.sel(lat = slice(latmin, latmax), lon = slice(lonmin, lonmax))
                    
                    # removing the climatology
                    da = RemoveClimatology(da)

                    # ------------
                    # Mean SSTA difference
                    # ------------
                    
                    # first calculating the average sst over the initial period
                    period1 = da.sel(time = slice(periodStart[0], periodStart[1])).mean(dim = 'time')

                    # second calculating the average sst over the final period
                    period2 = da.sel(time = slice(periodEnd[0], periodEnd[1])).mean(dim = 'time')

                    # third plotting the difference
                    diff = period2 - period1

                    # saving out the cumulative results to calculate the mean
                    if i == 0:
                        p1Cumulative = period1
                        p2Cumulative = period2
                        diffCumulative = diff
                    else:
                        p1Cumulative += period1
                        p2Cumulative += period2
                        diffCumulative += diff
                    
                    # ------------
                    # SD of SSTA
                    # ------------
                    SD1 = da.sel(time = slice(periodStart[0], periodStart[1])).std(dim = 'time')
                    SD2 = da.sel(time = slice(periodEnd[0], periodEnd[1])).std(dim = 'time')
                    SDDiff = da.sel(time = slice(periodStart[0], periodEnd[1])).std(dim = 'time')

                    # saving out cumulative results to calculate the mean
                    if i == 0:
                        p1SDCum = SD1
                        p2SDCum = SD2
                        SDDiffCum = SDDiff
                    else:
                        p1SDCum += SD1
                        p2SDCum += SD2
                        SDDiffCum += SDDiff
                    
                    i += 1
                    print(f'{model}: {i} / {N}')

        else:
            if i < N:

                # opening the dataset
                da = ModelInput(item).ds.ts.sel(lat = slice(latmin, latmax), lon = slice(lonmin, lonmax))

                # removing the climatology
                da = RemoveClimatology(da)

                # ------------
                # Mean SSTA difference
                # ------------

                # first calculating the average sst over the initial period
                period1 = da.sel(time = slice(periodStart[0], periodStart[1])).mean(dim = 'time')

                # second calculating the average sst over the final period
                period2 = da.sel(time = slice(periodEnd[0], periodEnd[1])).mean(dim = 'time')

                # third plotting the difference
                diff = period2 - period1

                # saving out the cumulative results to calculate the mean
                if i == 0:
                    p1Cumulative = period1
                    p2Cumulative = period2
                    diffCumulative = diff
                else:
                    p1Cumulative += period1
                    p2Cumulative += period2
                    diffCumulative += diff

                # ------------
                # SD of SSTA
                # ------------
                SD1 = da.sel(time = slice(periodStart[0], periodStart[1])).std(dim = 'time')
                SD2 = da.sel(time = slice(periodEnd[0], periodEnd[1])).std(dim = 'time')
                SDDiff = da.sel(time = slice(periodStart[0], periodEnd[1])).std(dim = 'time')

                # saving out cumulative results to calculate the mean
                if i == 0:
                    p1SDCum = SD1
                    p2SDCum = SD2
                    SDDiffCum = SDDiff
                else:
                    p1SDCum += SD1
                    p2SDCum += SD2
                    SDDiffCum += SDDiff

                i += 1
                print(f'{model}: {i} / {N}')

    dictPeriod1[model] = p1Cumulative / N
    dictPeriod2[model] = p2Cumulative / N
    dictDiff[model] = diffCumulative / N
    
    dictSD1[model] = p1SDCum / N
    dictSD2[model] = p2SDCum / N
    dictSDFull[model] = SDDiffCum / N

In [173]:
# saving out the files from this
MakeChangeDir(dirSSTPeriods)

for modelName, ds in dictPeriod1.items():
    pathName = dirSSTPeriods + '/'+ modelName + '_Period1.nc'
    ds.to_netcdf(pathName)

for modelName, ds in dictPeriod2.items():
    pathName = dirSSTPeriods + '/'+ modelName + '_Period2.nc'
    ds.to_netcdf(pathName)
    
for modelName, ds in dictDiff.items():
    pathName = dirSSTPeriods + '/'+ modelName + '_Diff.nc'
    ds.to_netcdf(pathName)

MakeChangeDir(dirSDPeriods)

for modelName, ds in dictSD1.items():
    pathName = dirSDPeriods + '/'+ modelName + '_Period1.nc'
    ds.to_netcdf(pathName)

for modelName, ds in dictSD2.items():
    pathName = dirSDPeriods + '/'+ modelName + '_Period2.nc'
    ds.to_netcdf(pathName)

for modelName, ds in dictSDFull.items():
    pathName = dirSDPeriods + '/'+ modelName + '_Full.nc'
    ds.to_netcdf(pathName)